In [1]:
import crypten
import torch
import sys
import os
from crypten.config import cfg
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# torch.set_default_tensor_type(torch.FloatTensor) 

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path

sys.path.append(parent_dir)


from privacy_utils import *
import pandas as pd

crypten.init()

/Users/jake/miniconda3/envs/Crypten/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crypten.cfg.config

{'communicator': {'verbose': False}, 'debug': {'debug_mode': False, 'validation_mode': False}, 'encoder': {'precision_bits': 16}, 'functions': {'max_method': 'log_reduction', 'exp_iterations': 8, 'reciprocal_method': 'NR', 'reciprocal_nr_iters': 10, 'reciprocal_log_iters': 1, 'reciprocal_all_pos': False, 'reciprocal_initial': None, 'sqrt_nr_iters': 3, 'sqrt_nr_initial': None, 'sigmoid_tanh_method': 'reciprocal', 'sigmoid_tanh_terms': 32, 'log_iterations': 2, 'log_exp_iterations': 8, 'log_order': 8, 'trig_iterations': 10, 'erf_iterations': 8}, 'mpc': {'active_security': False, 'provider': 'TFP', 'protocol': 'beaver'}, 'nn': {'dpsmpc': {'protocol': 'layer_estimation', 'skip_loss_forward': True, 'cache_pred_size': True}}}

Implementing linear EIG calc in crypten

In [4]:
n_host_sample = 2000 
sigma_error = 1
d = 10
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [5]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.std()) * (Y_host-Y_host.mean()) + sigma_error * torch.randn_like(Y_host)

In [6]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [7]:
sigma_error = 1
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)

In [8]:
X_host_no_T_2 = (torch.randn((n_host_sample,d)) @ A ) 
T_host_2 = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T_2 = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host_2 = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [9]:
sigma_error = 1
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}
bayesian_regression = BayesianLinearRegressionCrypt(prior_hyperparameters)
bayesian_regression.set_causal_index(10)

In [10]:
bayesian_regression.fit(X_host,Y_host)

{'posterior_mean': tensor([ 0.0759,  0.2098, -0.0218,  0.0189,  0.3198, -0.4664,  0.2373,  0.2579,
         -0.3803, -0.1173,  0.0177,  0.1092, -0.1203,  0.0455,  0.0765, -0.1415,
         -0.1087, -0.1111, -0.0222, -0.0344]),
 'posterior_cov_matrix': tensor([[ 9.7254e-01, -1.1801e-02,  6.5765e-03, -3.8058e-03, -1.9723e-02,
           2.5180e-03, -1.2882e-02, -1.6811e-02,  9.0497e-03, -2.1018e-02,
          -1.4134e-02, -6.3362e-03,  3.1640e-03, -1.5578e-03, -1.0779e-02,
           1.4501e-03, -7.0696e-03, -9.3083e-03,  4.9095e-03, -1.0913e-02],
         [-1.1801e-02,  9.7235e-01, -1.1566e-02, -5.6973e-03, -1.6439e-02,
          -1.5599e-03, -8.5832e-03, -1.1680e-04, -1.1509e-02, -2.2862e-02,
          -6.3392e-03, -1.3618e-02, -5.5426e-03, -2.6311e-03, -8.5291e-03,
          -4.7020e-04, -4.6188e-03, -1.0815e-03, -4.7218e-03, -1.1259e-02],
         [ 6.5765e-03, -1.1566e-02,  9.6835e-01,  3.5425e-03,  1.2065e-02,
          -1.2633e-03, -2.8886e-04, -1.1121e-02, -1.3734e-02, -3.3574e-0

In [24]:
bayesian_regression.samples_obs_EIG_comparison(X_host.double(),200,400)

predicted


200it [00:01, 166.67it/s]
200it [00:48,  4.08it/s]


{'torch': tensor(999.6143), 'crypt': tensor(999.6556)}

In [ ]:
y_pred = torch.randn_like(Y_host)
y_true = Y_host

(0.5*(((y_true-y_pred).T @ (y_true-y_pred))))

In [13]:
y_pred_crypt = crypten.mpc.MPCTensor(y_pred)
y_true_crypt = crypten.mpc.MPCTensor(y_true)

In [14]:
log_liklihood_crypt(y_true,y_pred,1,decrypt = False)

tensor(-2900.4487)

In [15]:
log_liklihood_crypt(y_true_crypt,y_pred_crypt,1,decrypt = True)

tensor(-2900.4209)

In [16]:
log_sum_exp_crypt(y_true,decrypt=False)

tensor(8.6229)

In [17]:
log_sum_exp_crypt(y_true_crypt,decrypt = True)

tensor(8.5706)

In [18]:
y_stack = torch.randn((10,len(y_pred_crypt)))
y_stack_crypt = crypten.mpc.MPCTensor(y_stack)

pred_list_torch = []
pred_list_crypt = []

for i in range(100):

    y_stack = torch.randn((10,len(y_pred)))
    y_stack_crypt = crypten.mpc.MPCTensor(y_stack)

    pred_list_torch.append((y_pred,y_stack))
    pred_list_crypt.append((y_pred_crypt,y_stack_crypt))

normal_samples = torch.randn((100,len(y_pred)))

In [19]:
calc_posterior_predictive_entropy_torch(pred_list_torch,normal_samples, 1)

100it [00:00, 5808.56it/s]


tensor(2924.4329)

In [20]:
calc_posterior_predictive_entropy_crypt(pred_list_crypt,normal_samples, 1,decrypt = True)

100it [00:10,  9.61it/s]


tensor(2924.3970)

In [21]:
compute_EIG_obs_from_samples_crypt(pred_list_crypt,normal_samples, 1)

100it [00:10,  9.68it/s]


tensor(2924.3970)

In [ ]:
log_posterior_predictive_torch(y_true, y_stack, sigma = 1,decrypt = False)

In [ ]:
log_posterior_predictive_crypten(y_true_crypt, y_stack_crypt, sigma = 1,decrypt = True)

tensor(-2870.8220)

In [ ]:
compute_EIG_obs_from_samples_crypt(pred_list_crypt,normal_samples, 1)

100it [00:10,  9.40it/s]


tensor(45.4246)

In [ ]:
compute_EIG_obs_from_samples_torch(pred_list_torch,normal_samples, 1)

100it [00:00, 3586.25it/s]


tensor(45.4600)

In [ ]:
compute_EIG_causal_from_samples_crypt(pred_list_crypt,pred_list_crypt,normal_samples,1)

100it [00:20,  4.84it/s]


tensor(0.)

In [ ]:
compute_EIG_causal_from_samples_torch(pred_list_torch,pred_list_torch,normal_samples,1)

100it [00:00, 2298.69it/s]


tensor(0.)

In [ ]:
def compute_EIG_causal_from_samples_crypt(pred_list_unpaired, pred_list_paired,normal_samples, sigma):
    """ " Function to calculate causal information gain"""
    # n_e = len(pred_list_unpaired[0][0])  # old len(pred_list_unpaired[0][0])
    # return calc_posterior_predictive_entropy(
    #     pred_list_unpaired, sigma
    # ) - calc_posterior_predictive_entropy(pred_list_paired, sigma)

    sample_list_crypten = crypten.mpc.MPCTensor(torch.zeros(len(min(pred_list_unpaired,pred_list_paired))))

    for i,((y_pred, y_pred_multiple_paired),(_,y_pred_multiple_unpaired)) in tqdm(enumerate(zip(pred_list_paired,pred_list_unpaired))):
        y_sample = y_pred + normal_samples[i]
        sample_list_crypten[i]  =   log_posterior_predictive_crypten(y_sample, y_pred_multiple_paired, sigma) - log_posterior_predictive_crypten(y_sample, y_pred_multiple_unpaired,sigma)
    return sample_list_crypten.mean().get_plain_text()

def compute_EIG_causal_from_samples_torch(pred_list_unpaired, pred_list_paired,normal_samples, sigma):
    """ " Function to calculate causal information gain"""
    # n_e = len(pred_list_unpaired[0][0])  # old len(pred_list_unpaired[0][0])
    # return calc_posterior_predictive_entropy(
    #     pred_list_unpaired, sigma
    # ) - calc_posterior_predictive_entropy(pred_list_paired, sigma)

    sample_list_crypten = torch.zeros(len(min(pred_list_unpaired,pred_list_paired)))

    for i,((y_pred, y_pred_multiple_paired),(_,y_pred_multiple_unpaired)) in tqdm(enumerate(zip(pred_list_paired,pred_list_unpaired))):
        y_sample = y_pred + normal_samples[i]
        sample_list_crypten[i]  =   log_posterior_predictive_torch(y_sample, y_pred_multiple_paired, sigma) - log_posterior_predictive_torch(y_sample, y_pred_multiple_unpaired,sigma)
    return sample_list_crypten.mean()

In [ ]:
def compute_EIG_obs_from_samples_crypt(pred_list,normal_samples, sigma):
    n_e = len(pred_list[0][0])
    return (calc_posterior_predictive_entropy_crypt(pred_list,normal_samples=normal_samples, sigma=sigma) - n_e / 2 * (
        1 + np.log(2 * np.pi * sigma**2)
    )).get_plain_text()

def compute_EIG_obs_from_samples_torch(pred_list,normal_samples, sigma):
    n_e = len(pred_list[0][0])
    return calc_posterior_predictive_entropy_torch(pred_list,normal_samples=normal_samples, sigma=sigma) - n_e / 2 * (1 + np.log(2 * np.pi * sigma**2))
        

In [ ]:
def compute_EIG_causal_from_samples_crypt(pred_list_unpaired, pred_list_paired, sigma):
    """ " Function to calculate causal information gain"""
    # n_e = len(pred_list_unpaired[0][0])  # old len(pred_list_unpaired[0][0])
    # return calc_posterior_predictive_entropy(
    #     pred_list_unpaired, sigma
    # ) - calc_posterior_predictive_entropy(pred_list_paired, sigma)
    n_e = len(pred_list_unpaired[0][0])  # old len(pred_list[0][0])
    covariance = cov.CovViaDiagonal(sigma**2 * np.ones(n_e))
    sample_list = []

    for (y_pred, y_pred_multiple_paired),(_,y_pred_multiple_unpaired) in tqdm(zip(pred_list_paired,pred_list_unpaired)):
        mvn = multivariate_normal(mean=y_pred, cov=covariance)
        y_sample = mvn.rvs()
        sample_list.append(
            log_posterior_predictive(
                y_sample, y_pred_multiple_paired, covariance
            )
            - log_posterior_predictive(
                y_sample, y_pred_multiple_unpaired, covariance
            ))
    return sum(sample_list) / len(sample_list)

In [ ]:
def calc_posterior_predictive_entropy_crypt(pred_list,normal_samples, sigma,decrypt = False):
    n_e = len(pred_list[0][0])  # old len(pred_list[0][0])

    sample_list_crypten = crypten.mpc.MPCTensor(torch.zeros(len(pred_list)))
    
    for i,(y_pred, y_pred_multiple) in tqdm(enumerate(pred_list)):

        y_sample = y_pred + normal_samples[i]
        sample_list_crypten[i] = log_posterior_predictive_crypten(y_sample,y_pred_multiple,sigma=sigma)
    if decrypt:
        return -sample_list_crypten.mean().get_plain_text()

    else:
        return -sample_list_crypten.mean()

def calc_posterior_predictive_entropy_torch(pred_list,normal_samples, sigma):

    sample_list_crypten = torch.zeros(len(pred_list))
    
    for i,(y_pred, y_pred_multiple) in tqdm(enumerate(pred_list)):

        y_sample = y_pred + normal_samples[i]
        sample_list_crypten[i] = log_posterior_predictive_torch(y_sample,y_pred_multiple,sigma=sigma)
    return -sample_list_crypten.mean()

In [ ]:
calc_posterior_predictive_entropy_crypt([([1],[1]),([2],[2])],1, 1)

In [ ]:
compare_obs_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),DP=True,epsilon=10)

{'EIG_obs_torch': 4.844079971313477,
 'EIG_obs_crypten': 4.8579254150390625,
 'EIG_obs_DP': 0.324850514992293}

In [ ]:
compare_caus_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),d,DP=True,epsilon=10)

{'EIG_caus_torch': 2.409762382507324,
 'EIG_caus_crypten': 2.408233642578125,
 'EIG_caus_DP': 4.11442765801425}

In [ ]:
compare_obs_sum_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),DP=True,epsilon=10)

In [ ]:
compare_caus_sum_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),d,DP=True,epsilon=10)

{'EIG_caus_torch': 0.7220475673675537,
 'EIG_caus_crypten': 0.589385986328125,
 'EIG_caus_DP': 38.9554611707243}

In [ ]:
def bairess_alg(encrypt,decrypted):
    M = encrypt # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in tqdm(range(N-1)):
        if decrypted[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if decrypted[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in (range(i+1,N)):
            for k in range(i+1,N):
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ).div(prev)
        prev = M[i][i]
    return sign * M[-1][-1]

In [ ]:
XX_host = X_host.T @ X_host + np.eye(X_host.shape[1])
XX_host_crypt = crypten.mpc.MPCTensor(XX_host)

In [ ]:
# a = bairess_alg(XX_host_crypt,XX_host)

## IDHP

In [25]:
n_host = 400
exp_parameters = {'number_of_candidate_sites': 20+1,
                'min_sample_size_cand': 200, 
                'max_sample_size_cand': 400, 
                'host_sample_size': n_host,
                'host_test_size': 2000,
                'outcome_function': None, 
                'std_true_y': 1, 
                'power_x': 1, 
                'power_x_t': 1,
                'coef_sample_width':5,
                "min_treat_group_size":20}
epsilon = 10

In [26]:
data_with_groundtruth, x, t, y = get_data("IDHP", "")
XandT = pd.concat([x,t], axis=1)
XandT = XandT.sample(n=10**4, replace=True, random_state=42)


candidate_sites = generating_random_sites_from(XandT, data_with_groundtruth, exp_parameters, added_T_coef=20, binary_outcome=False)  

host = candidate_sites[0][:n_host]
XandT_host, Y_host = torch.from_numpy(host.drop(columns=["Y"]).values), torch.from_numpy(host["Y"].values)
host_test = candidate_sites[0][n_host:]
causal_param_first_index = np.shape(XandT)[1]

candidate_sites = {key: value for key, value in candidate_sites.items() if key != 0}

cov = torch.eye(XandT_host.shape[1])

In [29]:
sigma_error = 1
# prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = torch.zeros(XandT_host.shape[1]), sigma_error,cov
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}
bayesian_regression = BayesianLinearRegressionCrypt(prior_hyperparameters)
bayesian_regression.set_causal_index(26)

In [30]:
cov.shape
bayesian_regression.fit(XandT_host, Y_host)

{'posterior_mean': tensor([-0.0343,  0.0498, -0.0581, -0.1656,  0.1223, -0.1185,  0.0850, -0.1454,
         -0.0850,  0.0945, -0.1785,  0.0389,  0.2419, -0.0862, -0.0331,  0.0529,
         -0.0343, -0.2307, -0.0343, -0.0240, -0.1955, -0.0653,  0.0000,  0.0338,
          0.0018,  0.0204,  0.0990, -0.0246, -0.1138,  0.0635,  0.2034, -0.1863,
          0.2246, -0.0905,  0.0000,  0.0990,  0.0000, -0.0905,  0.2419, -0.0284,
          0.0705,  0.0000,  0.0990, -0.1429,  0.0990, -0.0240,  0.0000,  0.0000,
          0.0000, -0.0905,  0.0000,  0.0000], dtype=torch.float64),
 'posterior_cov_matrix': tensor([[ 0.8394,  0.0206, -0.0613,  ...,  0.0060,  0.0000, -0.0000],
         [ 0.0206,  0.3548, -0.2308,  ..., -0.0339,  0.0000, -0.0000],
         [-0.0613, -0.2308,  0.3138,  ...,  0.0245,  0.0000, -0.0000],
         ...,
         [ 0.0060, -0.0339,  0.0245,  ...,  0.9002,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  1.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000

In [40]:
privacy_results_index = {
    'EIG_samples_torch':[],
    'EIG_samples_crypten':[]
}

for _, candidate in tqdm(candidate_sites.items()):
    
    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    results_dict = bayesian_regression.samples_obs_EIG_comparison(X_cand.double(),200,400)

    # obs_results_dict = compare_obs_sum_EIG_lin(XandT_host,X_cand,cov,DP=True,epsilon=epsilon)
    # caus_results_dict = compare_caus_sum_EIG_lin(XandT_host,X_cand,cov,DP=True,causal_param_first_index=26,epsilon=epsilon)

    privacy_results_index['EIG_samples_torch'].append( results_dict["torch"] )
    privacy_results_index['EIG_samples_crypten'].append( results_dict["crypt"] )

    # for key in caus_results_dict:
    #     privacy_results_index[key].append( caus_results_dict[key] )
    # print(obs_results_dict)
    # print(caus_results_dict)

  0%|          | 0/20 [00:00<?, ?it/s]

predicted


200it [00:00, 229.64it/s]
200it [00:48,  4.10it/s]
  5%|▌         | 1/20 [00:49<15:46, 49.79s/it]

predicted


200it [00:00, 236.87it/s]
200it [00:48,  4.11it/s]
 10%|█         | 2/20 [01:39<14:54, 49.72s/it]

...

200it [00:00, 231.54it/s]
200it [00:48,  4.08it/s]
 90%|█████████ | 18/20 [15:02<01:40, 50.28s/it]

predicted


200it [00:00, 236.63it/s]
200it [00:49,  4.06it/s]
 95%|█████████▌| 19/20 [15:53<00:50, 50.25s/it]

predicted


200it [00:00, 233.36it/s]
200it [00:49,  4.08it/s]
100%|██████████| 20/20 [16:43<00:00, 50.16s/it]


In [44]:
privacy_results_index

{'EIG_samples_torch': [tensor(416.7542),
  tensor(513.5315),
  tensor(570.7437),
  tensor(134.8114),
  tensor(549.7674),
  tensor(436.4828),
  tensor(516.9379),
  tensor(472.6981),
  tensor(477.7452),
  tensor(686.7145),
  tensor(460.5873),
  tensor(648.7919),
  tensor(729.6532),
  tensor(666.6307),
  tensor(612.0084),
  tensor(493.9106),
  tensor(754.9531),
  tensor(431.4104),
  tensor(477.1418),
  tensor(466.6683)],
 'EIG_samples_crypten': [tensor(1.1518e+09),
  tensor(3.9566e+08),
  tensor(1.8927e+09),
  tensor(-1.3695e+09),
  tensor(2.5606e+09),
  tensor(9.7547e+08),
  tensor(-4.2826e+09),
  tensor(8.0783e+08),
  tensor(1.1169e+09),
  tensor(-7.3348e+08),
  tensor(64155328.),
  tensor(7.2780e+08),
  tensor(3.6024e+09),
  tensor(-2.1990e+09),
  tensor(1.2309e+09),
  tensor(-3.3766e+09),
  tensor(-1.5381e+09),
  tensor(2.0037e+08),
  tensor(-1.7515e+09),
  tensor(-4.7925e+08)]}

In [41]:
import yaml
import os
from datetime import datetime

In [42]:
now = datetime.now()
date_time_str = now.strftime("%m-%d %H:%M:%S")
date_time_str.replace(" ","-")

'05-22-01:21:07'

In [ ]:
path_to='./'

In [43]:
with open(os.path.join(path_to,date_time_str.replace(" ","-")+"samples"+".metrics"), 'w') as f:
    yaml.dump(privacy_results_index, f)

In [ ]:
privacy_results_index = {
    'EIG_obs_torch':[],
    'EIG_obs_crypten':[],
    'EIG_obs_DP' : [],
    'EIG_caus_torch':[],
    'EIG_caus_crypten':[],
    'EIG_caus_DP':[]
}

for _, candidate in tqdm(candidate_sites.items()):

    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    obs_results_dict = compare_obs_sum_EIG_lin(XandT_host,X_cand,cov,DP=True,epsilon=epsilon)
    caus_results_dict = compare_caus_sum_EIG_lin(XandT_host,X_cand,cov,DP=True,causal_param_first_index=26,epsilon=epsilon)

    for key in obs_results_dict:
        privacy_results_index[key].append( obs_results_dict[key] )

    for key in caus_results_dict:
        privacy_results_index[key].append( caus_results_dict[key] )
    print(obs_results_dict)
    print(caus_results_dict)

In [ ]:
import scipy

In [ ]:
((np.array(privacy_results_index["EIG_caus_crypten"])-np.array(privacy_results_index["EIG_caus_DP"]))**2).mean()

1580.3901645546716

In [ ]:
scipy.stats.kendalltau(privacy_results_index["EIG_caus_crypten"],privacy_results_index["EIG_caus_DP"])

KendalltauResult(correlation=0.3636831673644744, pvalue=0.026667942877584984)

In [ ]:
now = datetime.now()
date_time_str = now.strftime("%m-%d %H:%M:%S")
date_time_str.replace(" ","-")

'05-17-10:05:58'

In [ ]:
with open(os.path.join(path_to,date_time_str.replace(" ","-")+"e="+str(epsilon)+".metrics"), 'w') as f:
    yaml.dump(privacy_results_index, f)

In [ ]:
results_dict["EIG_obs_DP"]

{'EIG_caus_DP': [404.6473315157036,
  403.77588404050664,
  400.0575454058906,
  401.9664029969986,
  399.8655207077245,
  404.44127426818625,
  397.7400546216603,
  398.0855507591822,
  395.6082049029357,
  401.2757817616515,
  400.02413435082804,
  402.08336540805317,
  399.8309840701867,
  400.14435270943017,
  400.44773561153977,
  402.1969765565662,
  398.06473937253116,
  401.4645357727816,
  396.34878771541287,
  399.71473320353795],
 'EIG_caus_crypten': [94.63760375976562,
  99.75857543945312,
  95.61798095703125,
  94.29258728027344,
  101.47918701171875,
  95.46809387207031,
  93.87242126464844,
  92.14152526855469,
  97.64878845214844,
  101.52085876464844,
  95.14735412597656,
  98.9700927734375,
  94.94633483886719,
  100.62191772460938,
  95.46809387207031,
  97.84297180175781,
  94.56358337402344,
  95.81211853027344,
  94.94633483886719,
  96.82487487792969],
 'EIG_caus_torch': [94.97021629571817,
  101.61620694999041,
  95.91348725583379,
  95.12429837149315,
  101.964

In [ ]:
import glob
import scipy

In [ ]:
n_runs = 3
err_samples_crypt = np.zeros(n_runs*20)
# err_samples_obs = np.zeros(n_runs*20)

# err_crypten_caus = np.zeros(n_runs*20)
# err_DP_caus = np.zeros(n_runs*20)

# rho_crypt_obs = np.zeros(n_runs)
# rho_crypt_caus = np.zeros(n_runs)

# rho_DP_obs = np.zeros(n_runs)
# rho_DP_caus = np.zeros(n_runs)

i = 0
for path in glob.glob(path_to):
    
    print(path)
    print(i)
    with open(path, "r") as f:
        results_dict = yaml.unsafe_load(f)

    err_samples_crypt[20*i:20*i+20] = ((((np.array(results_dict["EIG_samples_crypten"])-np.array(results_dict["EIG_obs_DP"]))/np.array(results_dict["EIG_obs_torch"])))**2).mean()
    # err_crypten_obs[20*i:20*i+20] = ((((np.array(results_dict["EIG_obs_torch"])-np.array(results_dict["EIG_obs_crypten"]))/np.array(results_dict["EIG_obs_torch"])))**2).mean()

    # err_DP_caus[20*i:20*i+20] = ((((np.array(results_dict["EIG_caus_torch"])-np.array(results_dict["EIG_caus_DP"]))/np.array(results_dict["EIG_obs_torch"])))**2).mean()
    # err_crypten_caus[20*i:20*i+20] = ((((np.array(results_dict["EIG_caus_torch"])-np.array(results_dict["EIG_caus_crypten"]))/np.array(results_dict["EIG_obs_torch"])))**2).mean()
    
    # rho_crypt_obs[i] = scipy.stats.kendalltau(np.array(results_dict["EIG_obs_torch"]),np.array(results_dict["EIG_obs_crypten"]))[0]
    # rho_crypt_caus[i] = scipy.stats.kendalltau(np.array(results_dict["EIG_caus_torch"]),np.array(results_dict["EIG_caus_crypten"]))[0]

    # rho_DP_obs[i] = scipy.stats.kendalltau(np.array(results_dict["EIG_obs_torch"]),np.array(results_dict["EIG_obs_DP"]))[0]
    # rho_DP_caus[i] = scipy.stats.kendalltau(np.array(results_dict["EIG_caus_torch"]),np.array(results_dict["EIG_caus_DP"]))[0]

    i+=1
    

In [ ]:
X_cand = torch.from_numpy(candidate_sites[1].drop(columns=["Y"]).values)
X_host = XandT_host
XX_cand = X_cand.T @ X_cand
X_host_plus_cov = X_host.T @ X_host + cov

XX_cand = X_cand.T @ X_cand
X_host_plus_cov = X_host.T @ X_host + cov
XX_cand_causal,X_host_plus_cov = XX_cand[causal_param_first_index:,causal_param_first_index:],X_host_plus_cov[causal_param_first_index:,causal_param_first_index:]
_,EIG_torch = torch.slogdet(XX_cand_causal + X_host_plus_cov )
# results_dict["EIG_caus_torch"] = EIG_torch.item()

X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov)
XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand_causal)
# EIG_crypten = logdet_Crypten((XX_cand_crypten + X_host_crypten)/len(X_host_crypten))
# EIG_crypten = EIG_crypten + (len(X_host_crypten))*np.log(len(X_host_crypten))
# EIG_crypten = EIG_crypten.get_plain_text()
# # EIG_crypten = logdet_Crypten(XX_cand_crypten + X_host_crypten).get_plain_text()
# results_dict["EIG_caus_crypten"] = EIG_crypten.item()

In [ ]:
L,D = chol_LD_Crypten(((XX_cand_crypten + X_host_crypten))/(len(X_host_crypten)))
c = D.get_plain_text()
c

tensor([10.6538,  9.1013,  2.5659,  3.7679,  7.8559, 11.6188, 10.2894,  2.3844,
         0.9200,  1.8434,  1.4379,  1.3063,  0.9156,  1.8771,  0.6737,  1.1941,
         0.4228,  1.5718,  0.1074,  1.1360,  0.5188,  1.2187,  0.1577,  0.2538,
         0.4782,  1.0884])

In [ ]:
D_log_enc = D.log()
(D_log_enc.sum() + len(D_log_enc)*np.log(len(X_host_crypten))).get_plain_text()

tensor(93.7979)

In [ ]:
EIG_crypten = logdet_Crypten((XX_cand_crypten + X_host_crypten)/len(X_host_crypten))
EIG_crypten = EIG_crypten + (len(X_host_crypten))*np.log(len(X_host_crypten))
EIG_crypten = EIG_crypten.get_plain_text()
EIG_crypten

tensor(93.7979)

In [ ]:
X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov)
XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand_causal)
EIG_crypten = logdet_Crypten((XX_cand_crypten + X_host_crypten)/len(X_host_crypten))
EIG_crypten = EIG_crypten + (len(X_host_crypten))*np.log(len(X_host_crypten))
EIG_crypten = EIG_crypten.get_plain_text()
EIG_crypten

tensor(93.7979)

In [ ]:
torch.linalg.ldl_factor((XX_cand_causal + (X_host_plus_cov))/(len(X_host_crypten)))[0].diag()

tensor([10.6538,  9.1049,  2.5721,  3.7707,  7.8567, 11.6305, 10.2932,  2.3923,
         0.9210,  1.8537,  1.4483,  1.3142,  0.9343,  1.8853,  0.7476,  1.1985,
         0.4526,  1.5894,  0.1266,  1.1368,  0.5195,  1.2198,  0.1578,  0.2547,
         0.4785,  1.1141], dtype=torch.float64)

In [ ]:
torch.linalg.(XX_cand_crypten + X_host_crypten)

In [ ]:
(XX_cand_crypten + X_host_crypten)/len(X_host_crypten)

In [ ]:
# scipy.stats.kendalltau(privacy_results_index["EIG_obs_torch"],privacy_results_index["EIG_obs_crypten"])

In [ ]:
scipy.stats.kendalltau(privacy_results_index["EIG_obs_torch"],privacy_results_index["EIG_obs_DP"])

KendalltauResult(correlation=-0.42857142857142855, pvalue=0.17886904761904762)

In [ ]:
scipy.stats.kendalltau(privacy_results_index["EIG_caus_torch"],privacy_results_index["EIG_caus_crypten"])

KendalltauResult(correlation=0.9285714285714285, pvalue=0.0003968253968253968)

In [ ]:
scipy.stats.kendalltau(privacy_results_index["EIG_caus_torch"],privacy_results_index["EIG_caus_DP"])

KendalltauResult(correlation=0.9999999999999998, pvalue=4.96031746031746e-05)

In [ ]:
# privacy_results_index

In [ ]:
X_cand = torch.from_numpy(candidate_sites[1].drop(columns=["Y"]).values)
X_host = XandT_host
XX_cand = X_cand.T @ X_cand
X_host_plus_cov = X_host.T @ X_host + cov

X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov)
XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand)

In [ ]:
torch.slogdet(XX_cand @ (torch.linalg.inv(X_host.T @ X_host + cov)) + torch.eye(X_host.shape[1]))

torch.return_types.slogdet(
sign=tensor(1., dtype=torch.float64),
logabsdet=tensor(33.2117, dtype=torch.float64))

In [ ]:
(torch.linalg.inv(X_host.T @ X_host + cov))

In [ ]:
X_host.T @ X_host + cov

In [ ]:
# _,EIG_torch = torch.slogdet(XX_cand + X_host_plus_cov)


# X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov)
# XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand)
# EIG_crypten = logdet_Crypten((XX_cand_crypten + X_host_crypten)/len(X_host_crypten))
# EIG_crypten = EIG_crypten + (len(X_host_crypten))*np.log(len(X_host_crypten))
# EIG_crypten = EIG_crypten.get_plain_text()

In [ ]:
torch.linalg.ldl_factor((XX_cand + (X_host_plus_cov))/(len(X_host_crypten)))[0].diag()

tensor([26.0385, 24.9869,  6.4785,  9.4489, 25.4464, 21.2184, 26.3486,  6.0280,
         1.8504,  5.1149,  4.4550,  3.2219,  2.4657,  5.1556,  2.1764,  2.7899,
         1.0656,  4.5261,  0.6062,  3.1365,  2.6383,  2.9467,  1.2423,  0.9740,
         1.7569,  1.9678,  4.5486,  4.5080,  1.2372,  1.6612,  4.2541,  4.1332,
         4.5958,  1.2259,  0.4055,  0.9586,  0.8002,  0.6709,  0.5280,  0.9208,
         0.4119,  0.6002,  0.2506,  0.8482,  0.0638,  0.5889,  0.3364,  0.6258,
         0.1008,  0.0970,  0.2468,  0.4970], dtype=torch.float64)

In [ ]:
L,D = chol_LD_Crypten(((X_cand_crypten + X_host_crypten))/(len(X_host_crypten)))
c = D.get_plain_text()
c

tensor([26.0385, 24.9867,  6.2116,  9.3439, 25.4461, 21.0718, 26.3146,  5.9456,
         1.8418,  5.0188,  4.3654,  3.1265,  2.3131,  5.1201,  1.3296,  2.7410,
         0.5140,  4.1803, -0.5827,  3.1584,  2.6378,  2.9510,  1.2583,  0.9779,
         2.0186,  2.5489,  4.6111,  4.4948,  1.2250,  1.7019,  4.2661,  4.1044,
         4.5725,  1.2283,  0.4050,  0.9675,  0.8370,  0.7172,  0.5478,  0.9367,
         0.4517,  0.6215,  0.2865,  0.8487,  0.0627,  0.5888,  0.3372,  0.6263,
         0.1008,  0.0971,  0.2511,  0.5187])

In [ ]:
XX_cand = X_cand.T @ X_cand
X_host_plus_cov = X_host.T @ X_host + cov
# X_host_plus_cov_inv = torch.linalg.inv(X_host_plus_cov)
_,EIG_torch = torch.slogdet((XX_cand + X_host_plus_cov))

X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov)
X_cand_crypten = crypten.mpc.MPCTensor(XX_cand)
# EIG_crypten = logdet_Crypten(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0])).get_plain_text()

In [ ]:
L,D = chol_LD_Crypten(((X_cand_crypten + X_host_crypten)/len(X_host_crypten)**(0.5)))
c = D.get_plain_text()

In [ ]:
((XX_cand @ X_host_plus_cov_inv - (X_cand_crypten @ X_host_crypten).get_plain_text())**2).mean()

tensor(6.3718e-06, dtype=torch.float64)

In [ ]:
D_log_enc = D.log()
(D_log_enc.sum() + len(D_log_enc)*np.log(len(X_host_crypten))).get_plain_text()

tensor(-7.5649e+08)

In [ ]:
np.log(c).sum() + len(c)*np.log(len(X_host_crypten))

tensor(149.2690)

In [ ]:
torch.slogdet((XX_cand + X_host_plus_cov)) 

torch.return_types.slogdet(
sign=tensor(1., dtype=torch.float64),
logabsdet=tensor(234.1446, dtype=torch.float64))

In [ ]:
b = torch.linalg.ldl_factor((XX_cand + X_host_plus_cov_inv) /len(X_host_plus_cov_inv))[0].diag()

In [ ]:
np.log(b).sum()+len(b)*np.log(100)

tensor(183.1560, dtype=torch.float64)

In [ ]:
torch.log(torch.abs(a)[0].diag())

In [ ]:
torch.linalg.cholesky(XX_cand + X_host_plus_cov_inv +torch.eye(XX_cand.shape[0]))

tensor([[14.7693,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.3290, 14.8053,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6013, 12.2576,  7.8191,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.1361,  0.1618, -0.0268,  ...,  1.4702,  0.0000,  0.0000],
        [ 0.2039,  0.0804, -0.0923,  ..., -0.0915,  1.6310,  0.0000],
        [ 1.8290,  0.0403,  0.1559,  ..., -0.2588, -0.1564,  2.6115]],
       dtype=torch.float64)

In [ ]:
torch.prod(a[0].diag())

tensor(-30831.2644, dtype=torch.float64)

In [ ]:
def choleski(a):
    n = len(a)
    for k in range(n):
        a[k,k] = (a[k,k] -  a[k,0:k] @ a[k,0:k]).sqrt()
        for i in range(k+1,n):
            a[i,k] = (a[i,k] - a[i,0:k] @ a[k,0:k])/a[k,k]
    for k in range(1,n): a[0:k,k] = 0.0
    return a

In [ ]:
a = choleski(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]))

In [ ]:
a_correct = choleski(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]))

In [ ]:
plain_text = a.get_plain_text()

In [ ]:
torch.diag(plain_text)

tensor([ 1.3731e+00,  1.0809e+00,  1.1151e+00,  1.0815e+00,  1.0622e+00,
         1.0787e+00,  8.6667e-01,  1.0089e+00,  1.0988e+00,  9.0668e-01,
         1.0940e+00,  1.1447e+00,  1.1266e+00,  1.0273e+00, -2.0645e+09,
         1.9799e+09, -2.3924e+08, -7.5471e+08, -1.2316e+09,  1.2787e+09,
        -7.4276e+08, -1.7109e+09, -2.0575e+09, -1.2524e+09,  2.3476e+08,
        -3.9914e+08, -2.7453e+08,  6.4315e+07, -2.7571e+08, -1.7941e+09,
        -5.8454e+08, -1.8026e+09, -2.0025e+09,  2.7680e+08, -1.0890e+09,
        -9.9119e+08,  1.2683e+09,  5.8616e+08, -1.1281e+09,  4.5381e+08,
         1.2214e+09, -1.5685e+09,  2.1241e+09, -1.5466e+09, -1.7308e+09,
        -4.6608e+08, -6.3313e+08, -5.2956e+08, -5.5944e+08, -5.4547e+08,
         1.4762e+09,  1.7846e+09])

In [ ]:
X_cand_crypten.sq

tensor([ 1.8814,  1.1676,  1.2472,  1.1714,  1.1313,  1.1658,  0.7538,  1.0200,
         1.2116,  0.8259,  1.2012,  1.3140,  1.2728,  1.0577, -2.5659,  1.4263,
         1.3614,  1.7462,  1.2629,  1.2134,  1.1926,  1.2353,  1.0685,  1.2052,
         1.0732,  1.2899,  0.8869,  1.2257,  1.2417,  1.2375,  1.3422,  1.2943,
         1.2717,  1.2162,  1.2337,  1.1995,  1.2212,  1.2369,  1.2307,  1.1978,
         1.1960,  1.2608,  1.1825,  1.1426,  1.1508,  1.1650,  1.2741,  1.2078,
         1.1424,  1.2924,  1.1560,  1.1995])

In [ ]:
((D.get_plain_text()-D_true)**2).mean()

tensor(1.9748e-05)

In [ ]:
L_true,D_true = chol_LD_torch(XX_cand @ X_host_plus_cov_inv +torch.eye(XX_cand.shape[0]))

In [ ]:
L,D = chol_LD_Crypten((X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0])))
D.get_plain_text()

In [ ]:
a = bairess_alg(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]),(XX_cand @ X_host_plus_cov_inv + torch.eye(XX_cand.shape[0])))
a

100%|██████████| 51/51 [22:10<00:00, 26.08s/it]


MPCTensor(
	_tensor=73350087966301
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)

In [ ]:
torch.log(a.get_plain_text())

tensor(20.8359)

In [ ]:
(a.log()).get_plain_text()

tensor(2.5102e+10)

In [ ]:
def householder(a):
    n = len(a)
    for k in range(n-2):
        u = a[k+1:n,k]
        uMag = math.sqrt(np.dot(u,u))
        if u[0] < 0.0: uMag = -uMag
        u[0] = u[0] + uMag
        h = np.dot(u,u)/2.0
        v = np.dot(a[k+1:n,k+1:n],u)/h
        g = np.dot(u,v)/(2.0*h)
        v = v - g*u
        a[k+1:n,k+1:n] = a[k+1:n,k+1:n] - np.outer(v,u)  \
                         -np.outer(u,v)
    return np.diagonal(a),np.diagonal(a,1)

In [ ]:
d_plain = D.get_plain_text()

In [ ]:
torch.abs(d_plain)

tensor([ 5.6834,  7.4158, -0.2931, 26.2756,  1.4323,  2.0508,  2.2263,  2.4757,
         2.1785,  2.3619,  2.5277,  0.4344, -1.3256, 20.7987, 26.1783,  1.2382,
         0.3080, -2.7296,  8.1274,  1.6298,  2.5500,  0.7549,  2.6070,  1.8040,
         1.6967,  1.1568,  0.9444,  1.0805,  1.0641,  1.0747,  1.0839,  1.0892,
         1.1012,  1.0755,  1.1101,  1.0678,  1.0797,  1.0913,  1.0876,  1.0640,
         1.1266,  1.0649,  1.0638,  1.0567,  1.0213,  1.0413,  1.0454,  1.0862,
         1.0321,  1.2468,  1.0170,  1.0797])

In [ ]:
L.get_plain_text()

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 4.3465e-01,  1.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.5124e-01,  5.5630e-01,  1.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-1.9073e-03,  6.5613e-04, -4.5166e-03,  ...,  1.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.8311e-04, -7.6294e-05,  7.7820e-04,  ..., -1.6479e-03,
          1.0000e+00,  0.0000e+00],
        [-5.2643e-03,  1.6479e-03, -4.6844e-03,  ..., -1.8845e-02,
         -7.0801e-03,  1.0000e+00]])

In [ ]:
torch.linalg.lu_factor(XX_cand @ X_host_plus_cov_inv + torch.eye(XX_cand.shape[0]))

torch.return_types.linalg_lu_factor(
LU=tensor([[ 5.6900e+00,  1.5797e+01,  7.4488e-01,  ..., -5.2595e-01,
          2.4484e+00,  5.8242e-01],
        [ 9.9708e-01, -5.0926e+00, -5.2313e-01,  ...,  2.6421e-01,
         -7.7259e-01, -2.3422e-01],
        [ 2.5166e-01, -1.5067e-01,  2.0935e+00,  ..., -4.6215e-01,
         -3.4607e-01,  3.5035e-02],
        ...,
        [-1.8953e-03, -5.9370e-03, -1.9955e-04,  ...,  1.2441e+00,
         -1.3452e-02, -8.8851e-03],
        [ 1.9717e-04,  6.4983e-04, -3.1833e-05,  ..., -2.0259e-03,
          1.0179e+00,  1.1374e-03],
        [-5.2064e-03, -1.5931e-02, -1.9270e-03,  ..., -2.3199e-02,
         -4.9377e-03,  1.0888e+00]], dtype=torch.float64),
pivots=tensor([15, 15,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52],
       dtype=torch.int32))

In [ ]:
with crypten.mpc.ConfigManager("reciprocal_nr_iters", 100):
    XX_cand_caus = X_cand.T @ X_cand
    X_host_plus_cov = X_host.T @ X_host + cov
    XX_cand_causal,X_host_plus_cov = XX_cand_caus[causal_param_first_index:,causal_param_first_index:],X_host_plus_cov[causal_param_first_index:,causal_param_first_index:]
    X_host_plus_cov_inv_caus = torch.linalg.inv(X_host_plus_cov)
    _,EIG_torch_caus = torch.slogdet(XX_cand_causal @ X_host_plus_cov_inv_caus + torch.eye(X_host_plus_cov_inv_caus.shape[0]))

    X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov_inv)
    XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand_causal)
    EIG_crypten_caus = logdet_Crypten(XX_cand_crypten @ X_host_crypten + torch.eye(X_host_plus_cov_inv_caus.shape[0])).get_plain_text()

In [ ]:
print(EIG_torch_caus,EIG_crypten_caus)

tensor(12.4140, dtype=torch.float64) tensor(11.0452)


In [ ]:
D_plain = D.get_plain_text()

In [ ]:
torch.log(torch.abs(D_plain)).sum()

tensor(20.7083)

In [ ]:
torch.log(D_plain).sum()

tensor(nan)

## Misc

In [ ]:
# if type(X_host) is torch.Tensor:
# 
cov = np.eye(X_host.shape[1])
y = np.zeros(len(X_host))

Z = get_diff_private_version(X_host,y)

XX_host = Z["XX"]

_,logdet_1 = np.linalg.slogdet(X_cand.T @ X_cand + XX_host + cov)
_,logdet_2 = np.linalg.slogdet(XX_host + cov)

In [ ]:
X = np.array(X_host)
y = np.array(Y_host)

In [ ]:
y .dot(y)

4069.0527

In [ ]:
# sufficient statistics
S = {'XX': X.T.dot(X),
        'Xy': X.T.dot(y),
        'yy': y .dot(y)
        }

posteriors = {}

# if 'non-private' in methods:
#     posteriors['non-private'] = run_non_private(model_prior_params, S, N)


#posteriors['naive'] = run_naive(model_prior_params, S, N, sensitivity_x, sensitivity_y, epsilon)

In [ ]:
X = np.array(X_host)
y = np.array(Y_host)

sensitivity_x, sensitivity_y = get_sensativitiy(X,y)

epsilon = 1

In [ ]:
data_dim = S['XX'].shape[0]

XX_comps = data_dim * (data_dim + 1) / 2  # upper triangular, not counting last column which is X
X_comps = data_dim  # last column
Xy_comps = data_dim
yy_comps = 1
sensitivity = XX_comps * sum(sensitivity_x[:-1]) ** 2 \
                + X_comps * sum(sensitivity_x[:-1]) \
                + Xy_comps * sum(sensitivity_x[:-1]) * sensitivity_y \
                + yy_comps * sensitivity_y ** 2

Z = {key: np.random.laplace(loc=val, scale=sensitivity / epsilon) for key, val in S.items()}

# symmetrize Z_XX since we only want to add noise to upper triangle
Z['XX'] = symmetrize(Z['XX'])

Z['X'] = Z['XX'][:, 0][:, None]

In [ ]:
Z["X"].mean()

2205.008211286303

In [ ]:
X_host.mean()

-8.1423976e-05

In [ ]:
Z.keys()

dict_keys(['XX', 'Xy', 'yy', 'X'])

In [ ]:
priv_XX = torch.tensor(Z["XX"])

In [ ]:
(X_host.T @ X_host).mean()

0.018392561

In [ ]:
type(X_host)

numpy.ndarray

In [ ]:
priv_XX.mean()

tensor(455.8781, dtype=torch.float64)

In [ ]:
np.linalg.slogdet(priv_XX +np.eye(X_host.shape[1]) )

(-1.0, 186.1891397270062)

In [ ]:
torch.linalg.slogdet(X_host.T @ X_host +np.eye(X_host.shape[1]))

In [ ]:
priv_X - X_host.T @ X_host

tensor([[ 4.1798, -0.1939,  2.3292,  5.1394, -4.4555,  1.1445,  1.1193, -4.0168,
          1.9040,  2.1432,  2.0106, -0.1350,  1.0632,  2.4952, -2.1895,  0.5696,
          0.4913, -1.8778,  0.8573,  0.9238],
        [-0.1939,  7.7291, -3.9290, -1.8749,  2.1200, -0.8760,  2.3354, -1.7791,
         -4.5772, -2.4300, -0.1276,  3.8359, -1.8889, -0.8781,  1.1507, -0.5232,
          1.0409, -0.9258, -2.3725, -1.2714],
        [ 2.3292, -3.9290,  6.9166,  4.7498,  0.2810,  1.2035, -3.9490,  0.9760,
          2.9695,  2.4714,  1.0389, -1.9428,  3.3068,  2.2264,  0.1712,  0.6588,
         -2.0021,  0.6660,  1.3276,  1.1224],
        [ 5.1394, -1.8749,  4.7498,  8.8634, -5.6383,  1.4736,  0.3408, -4.7306,
          4.0217,  3.2184,  2.4766, -0.9318,  2.1888,  4.2901, -2.7647,  0.7033,
          0.1061, -2.2913,  1.8564,  1.4242],
        [-4.4555,  2.1200,  0.2810, -5.6383,  7.8555, -3.6295, -2.4313,  2.7517,
         -3.7156, -4.4249, -2.1930,  1.1223,  0.1595, -2.7562,  3.9227, -1.7315,
      

In [ ]:
X[:,0].max()

0.061485384

In [ ]:
X[:,0].min()

-0.059155844

In [ ]:
def get_sensativitiy(X,y):

    sens_x = np.zeros(X.shape[1])

    for i in range((X.shape[1])):
        sens_x[i] = X[:,i].max() - X[:,i].min()
    
    sens_y = y.max() - y.min()

    return sens_x,sens_y